## Код для интерпретации эмодзи 

Позволяет увеличить эффективность предсказаний при Named Entity-Oriented Sentiment Analysis

In [13]:
import pandas as pd
import re

In [14]:
df_names = pd.read_pickle('sentiment_texts.pickle')
df_names.head()

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False


In [15]:
emoji = pd.read_excel('emoji_ru.xlsx')
emoji.head()

,Unnamed: 0,Emoji_ID,Emoji,Unicode_Name,Ru_Name,Class,Total_Occurrence,Negativity_Occurrence_N(Negative),Neutrality_Occurrence_N(Neutral),Positivity_Occurrence_N(Positive),Negativite_Probability_P(Negative),Neutral_Probability_P(Neutral),Positive_Probability_P(Positive),Sentiment_Score_S,label
0,1,b'\xf0\x9f\x98\x82',😂,Face With Tears Of Joy,Лицо со слезами радости,Facial Expression,25908,8106,2638,15164,0.312876,0.101822,0.585302,0.272426,2
1,2,b'\xf0\x9f\x92\x94',💔,Broken Heart,Разбитое сердце,Heart,18564,17781,342,441,0.957822,0.018423,0.023756,-0.934066,0
2,3,b'\xe2\x9d\xa4',❤,Red Heart,Красное сердце,Heart,15876,2701,1573,11602,0.170131,0.099080,0.730789,0.560658,2
3,4,b'\xf0\x9f\x98\xad',😭,Loudly Crying Face,Громко плачущее лицо,Facial Expression,12318,9525,1617,1176,0.773259,0.131271,0.095470,-0.677789,0
4,5,b'\xf0\x9f\x98\x8d',😍,Smiling Face With Heart-Eyes,Улыбающееся лицо с глазами-сердечками,Facial Expression,6815,198,493,6124,0.029054,0.072340,0.898606,0.869552,2


In [16]:
def process_emoji_names(df_names):
    """
    Process emoji names in the 'MessageText' column of the DataFrame by replacing emoji symbols
    with their corresponding names and removing extra spaces.

    Parameters:
    - df_names (pd.DataFrame): DataFrame containing the 'MessageText' column with text to process.

    Returns:
    - None: Updates the 'MessageText' column in the input DataFrame in place.
    """

    emoji_mapping = dict(zip(emoji['Emoji'], emoji['Ru_Name']))
    emoji_pattern = re.compile('|'.join(map(re.escape, emoji_mapping.keys())))

    def replace_emoji_names(text):
        """
        Replace emoji symbols in the text with their corresponding names using the emoji_mapping.

        Parameters:
        - text (str): Input text containing emoji symbols to process.

        Returns:
        - str: Processed text with emoji symbols replaced by names.
        """
        def replace(match):
            return " " + emoji_mapping[match.group(0)] + " "

        return emoji_pattern.sub(replace, text)

    df_names['MessageText'] = df_names['MessageText'].apply(replace_emoji_names)

    remove_space = lambda text: re.sub(r'\s{2,}', ' ', text)
    df_names['MessageText'] = df_names['MessageText'].apply(remove_space)

In [18]:
process_emoji_names(df_names)

In [19]:
df_names.head()

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,Предупреждение ️ Россия #SELG #дивиденд сд Се...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются Огонь Акции и инвестиции ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются Огонь Акции и инвестиции ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False


In [12]:
df_names.to_excel('sentiment_texts_with_emojis.xlsx', index=False)